<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/results/Distribution_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)
print('cuda: ', torch.cuda.is_available())

2.1.0+cu121
cuda:  True


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet
! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu121/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet
! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet

In [3]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [4]:
from scripts.func_utils import make_path, convert_with_qed_sa, get_min, \
                                plot_loss, plot_performance

from scripts.eval_utils import *
from scripts.preprocess_mols import *
from scripts.model_architecture import *
from scripts.dataset import *
from scripts.train import *
import yaml
import pandas as pd
from scripts.get_vocab import *

from tdc.single_pred import ADME
from tdc.single_pred import Tox
from scripts.CONSTANT import *
from scripts.yaml_utils import *

print('VOCAB TYPE:', VOCAB_TYPE)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
VOCAB TYPE: smiles


In [5]:
names_D3

['BBB_Martins', 'PPBR_AZ', 'VDss_Lombardo']

In [6]:
from scripts.TRAIN import *
# train_mt(name, model_type, folder_name, repeat_time, retrain, uw,
#              clean_mol_=False)

In [8]:
for model_type in ['MLP', 'AttentiveFP', 'GIN', 'RNN']:
    p = train_mt(names_D3, model_type, 'D3/uw', repeat_time=3, retrain=False, uw=True)


Run # 0 for MLP MT	 | save dir:  D3/uw/MLP_MT_0 | 	--> pre data loaded

Run # 1 for MLP MT	 | save dir:  D3/uw/MLP_MT_1 | 	--> pre data loaded

Run # 2 for MLP MT	 | save dir:  D3/uw/MLP_MT_2 | 	--> pre data loaded
repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       w_acc      |       prec      |       recall      |       sp      |       f1      |       auc      |       mcc      |       ap      
	&0.778$\pm$0.034  &0.570$\pm$0.075  &0.776$\pm$0.032  &0.991$\pm$0.008  &0.149$\pm$0.159  &0.870$\pm$0.017  &0.820$\pm$0.036  &nan$\pm$nan  &0.926$\pm$0.016  
 idx 1: &0.825            &0.675            &0.820            &0.980            &0.369            &0.893            &0.861            &0.489            &0.944            

******************** PPBR_AZ ******************** 
	|       mae      |       mse      |       rmse      |       r2      |       pcc      |       spearman      
	&10.897$\pm$0.367  &216.091$\pm$8.9

In [9]:
from scripts.MUE import *

In [ ]:
names = names_D3
folder_name  = 'D3/uw'
model_dict={'MLP': 1,
            'AttentiveFP':1,
            'GIN':0,
            'RNN':2
            }
repeat_time = 3
retrain = False
trn, val, tst = collect_data(names)

trn_list = process_ensemble_v1(trn, folder_name, model_dict)
val_list = process_ensemble_v1(val, folder_name, model_dict)
tst_list = process_ensemble_v1(tst, folder_name, model_dict)

In [ ]:

p_list = []

for task_num in range(len(names)):
    perfs = []
    task_name = names[task_num]

    config = get_config('MUE', task_name)

    batch_size = config['batch_size']
    params_ = {'batch_size': batch_size, 'shuffle': True,
                'drop_last': False, 'num_workers': 0}

    param_t = {'batch_size': batch_size, 'shuffle': False,
                'drop_last': False, 'num_workers': 0}

    trn_l = DataLoader(ensemble_dataset(trn_list[task_num], task_name), **params_)
    val_l = DataLoader(ensemble_dataset(val_list[task_num], task_name), **params_)
    tst_l = DataLoader(ensemble_dataset(tst_list[task_num], task_name), **param_t)

    for i in range(repeat_time): # train MUE 3 times
        config_path = folder_name + '/MUE_' + task_name  + '_' +  str(i) + '.yml'
        model_path  = folder_name + '/MUE_' + task_name  + '_' +  str(i) + '.pt'

        config['config_path'] = config_path
        config['model_path'] = model_path
        nofile = False
        try:
            with open(config['config_path'], 'r') as f: data = yaml.safe_load(f)
            if data != None:
                p = yml_report(data); print('--> pre data loaded')
        except:
            print(f"cannot open {config['config_path']}, retrain")
            nofile = True

        if nofile or retrain:
            models = PRED(**config)
            p = models.train(trn_l, val_l, tst_l)

        perfs.append(p)
    p_list.append(perfs)


In [12]:
for name in names_D3:
    # is_r = names_dict[name]
    perf = []
    for i in range(3):
        yml_file = f'D3/uw/MUE_{name}_{i}.yml'
        p = yml_report(yml_file, ver=False)
        perf.append(p)
    eval_perf_list(perf, name, {})

repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       w_acc      |       prec      |       recall      |       sp      |       f1      |       auc      |       mcc      |       ap      
	&0.865$\pm$0.002  &0.773$\pm$0.005  &0.872$\pm$0.003  &0.959$\pm$0.002  &0.586$\pm$0.012  &0.914$\pm$0.001  &0.906$\pm$0.001  &0.619$\pm$0.007  &0.962$\pm$0.000  
 idx 1: &0.867            &0.780            &0.876            &0.957            &0.602            &0.915            &0.907            &0.627            &0.963            

repeated num # 3 idx 2 has the lowest loss
******************** PPBR_AZ ******************** 
	|       mae      |       mse      |       rmse      |       r2      |       pcc      |       spearman      
	&9.417$\pm$0.299  &164.865$\pm$4.432  &12.839$\pm$0.172  &0.319$\pm$0.018  &0.596$\pm$0.015  &0.646$\pm$0.015  
 idx 2: &9.449            &161.281            &12.700            &0.334            &0.617   